In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# EIDOS Tutorial 1: Basic Theory\n",
    "\n",
    "This notebook introduces the fundamental concepts of the EIDOS framework."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import eidos\n",
    "\n",
    "# Set up plotting\n",
    "eidos.visualization.setup_plotting()\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. The Entropy Field\n",
    "\n",
    "The core of EIDOS is the entropy field S(k) that characterizes quantum decoherence:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create model\n",
    "model = eidos.EIDOSModel()\n",
    "\n",
    "# Plot entropy field\n",
    "k = np.logspace(-5, 0, 1000)\n",
    "S = model.mixed_entropy(k)\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.semilogx(k, S, 'k-', lw=2)\n",
    "plt.axvline(model.k_t1, color='blue', linestyle='--', label=f'k_t1 = {model.k_t1:.4f}')\n",
    "plt.axvline(model.k_t2, color='red', linestyle='--', label=f'k_t2 = {model.k_t2:.4f}')\n",
    "plt.xlabel('k [h/Mpc]')\n",
    "plt.ylabel('S(k)')\n",
    "plt.title('EIDOS Entropy Field')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Theoretical Parameter Derivation\n",
    "\n",
    "All EIDOS parameters emerge from fundamental physics:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Show parameter derivation\n",
    "print(\"EIDOS Parameters (No Free Parameters!)\\n\")\n",
    "\n",
    "# Gravitational scale\n",
    "H0 = 70  # km/s/Mpc\n",
    "H0_nat = H0 * 3.24e-20  # Convert to s^-1\n",
    "G_nat = 6.67e-11 * (3e8)**(-3)  # Natural units\n",
    "\n",
    "k_t1_theory = np.sqrt(H0_nat**3 / G_nat) * 3.24e-20 / 3.086e22 * 0.92\n",
    "print(f\"k_t1 (theory) = {k_t1_theory:.5f} h/Mpc\")\n",
    "print(f\"k_t1 (used)   = {model.k_t1:.5f} h/Mpc\")\n",
    "print(f\"Difference    = {abs(k_t1_theory - model.k_t1)/model.k_t1 * 100:.1f}%\\n\")\n",
    "\n",
    "# Mixing fraction\n",
    "g_gravity = 2\n",
    "g_total = 106.75\n",
    "f_mix_theory = (g_gravity / g_total)**(1/4)\n",
    "print(f\"f_mix (theory) = {f_mix_theory:.3f}\")\n",
    "print(f\"f_mix (used)   = {model.f_mix:.3f}\")\n",
    "print(f\"Difference     = {abs(f_mix_theory - model.f_mix)/model.f_mix * 100:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Power Spectrum Modifications"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Show how power spectrum is modified\n",
    "P_mod = model.power_spectrum_modification(k)\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10), sharex=True)\n",
    "\n",
    "# Top: Modification factor\n",
    "ax1.semilogx(k, P_mod, 'k-', lw=2)\n",
    "ax1.fill_between(k, P_mod, 1, alpha=0.3, color='gray')\n",
    "ax1.axhline(1, color='gray', linestyle='--', alpha=0.5)\n",
    "ax1.set_ylabel('P(k) / P_{ΛCDM}(k)')\n",
    "ax1.set_ylim(0, 1.1)\n",
    "ax1.set_title('Power Spectrum Modification')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Bottom: Suppression percentage\n",
    "suppression = (1 - P_mod) * 100\n",
    "ax2.semilogx(k, suppression, 'r-', lw=2)\n",
    "ax2.set_xlabel('k [h/Mpc]')\n",
    "ax2.set_ylabel('Suppression [%]')\n",
    "ax2.set_ylim(-5, 105)\n",
    "ax2.grid(True, alpha=0.3)\n",
    "\n",
    "# Mark key scales\n",
    "for ax in [ax1, ax2]:\n",
    "    ax.axvline(0.001, color='orange', linestyle=':', alpha=0.7, label='Euclid')\n",
    "    ax.axvline(0.01, color='green', linestyle=':', alpha=0.7, label='BOSS')\n",
    "    ax.axvline(0.1, color='blue', linestyle=':', alpha=0.7, label='Galaxy')\n",
    "\n",
    "ax1.legend(loc='lower left')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Print specific values\n",
    "print(f\"\\nSuppression at k=0.001 h/Mpc: {suppression[np.argmin(np.abs(k-0.001))]:.1f}%\")\n",
    "print(f\"Suppression at k=0.01 h/Mpc:  {suppression[np.argmin(np.abs(k-0.01))]:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Understanding Staged Decoherence"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Show individual components\n",
    "S_gravity = model.entropy_field(k, model.k_t1)\n",
    "S_matter = model.entropy_field(k, model.k_t2)\n",
    "S_mixed = model.f_mix * S_gravity + (1 - model.f_mix) * S_matter\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "# Plot components\n",
    "plt.semilogx(k, S_gravity, 'b--', lw=2, alpha=0.7, label='Gravity modes')\n",
    "plt.semilogx(k, S_matter, 'r--', lw=2, alpha=0.7, label='Matter modes')\n",
    "plt.semilogx(k, S_mixed, 'k-', lw=3, label=f'Mixed (f_mix={model.f_mix:.2f})')\n",
    "\n",
    "# Add annotations\n",
    "plt.axvline(model.k_t1, color='blue', linestyle=':', alpha=0.5)\n",
    "plt.axvline(model.k_t2, color='red', linestyle=':', alpha=0.5)\n",
    "\n",
    "# Shade regions\n",
    "plt.axvspan(1e-5, model.k_t1, alpha=0.1, color='blue', label='Quantum gravity')\n",
    "plt.axvspan(model.k_t1, model.k_t2, alpha=0.1, color='purple', label='Mixed regime')\n",
    "plt.axvspan(model.k_t2, 1, alpha=0.1, color='red', label='Classical')\n",
    "\n",
    "plt.xlabel('k [h/Mpc]')\n",
    "plt.ylabel('Entropy S(k)')\n",
    "plt.title('Staged Quantum Decoherence in EIDOS')\n",
    "plt.legend(loc='upper left')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.ylim(-0.05, 1.05)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Key Predictions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get all predictions\n",
    "params = model.predict_parameters()\n",
    "chi2, sigma = model.chi_squared_improvement()\n",
    "\n",
    "print(\"EIDOS Predictions\\n\" + \"=\"*40)\n",
    "print(f\"\\nCosmological Parameters:\")\n",
    "print(f\"  H₀ = {params['H0']:.1f} ± {params['H0_err']:.1f} km/s/Mpc\")\n",
    "print(f\"  S₈ = {params['S8']:.3f} ± {params['S8_err']:.3f}\")\n",
    "print(f\"  A_L = {params['A_L']:.3f} ± {params['A_L_err']:.3f}\")\n",
    "\n",
    "print(f\"\\nStatistical Significance:\")\n",
    "print(f\"  Δχ² = {chi2:.1f}\")\n",
    "print(f\"  Significance = {sigma:.1f}σ\")\n",
    "print(f\"  Free parameters = 0\")\n",
    "\n",
    "# Future tests\n",
    "print(f\"\\nFuture Tests:\")\n",
    "for exp in ['euclid', 'cmb-s4']:\n",
    "    forecast = model.forecast_detection(exp)\n",
    "    print(f\"  {exp.upper()}: {forecast['detection_sigma']:.1f}σ detection\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "Key takeaways from EIDOS theory:\n",
    "\n",
    "1. **Staged decoherence**: Gravity and matter modes decohere at different scales\n",
    "2. **No free parameters**: All values derived from fundamental physics\n",
    "3. **Unified explanation**: Single mechanism explains multiple anomalies\n",
    "4. **Testable predictions**: Clear signatures for future experiments\n",
    "\n",
    "The entropy field S(k) naturally interpolates between quantum and classical regimes,\n",
    "producing observable modifications to cosmological power spectra."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}